# 十種猴子影像辨識

## 導入模組

In [3]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using Statistics
using Images
using JLD

## 載入資料

### 載入訓練資料

In [9]:
monkeys = Array{Float64}(undef, 1097, 72, 128, 3)
mon_key = 1
pbase = "training/training/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = imresize(img, (288, 512))
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(72, 128, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:72)
                for k in (1:128)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        monkeys[mon_key, :, :, :] = c2
        mon_key += 1
    end
end

┌ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1260


### 儲存訓練資料

In [10]:
save("monkey_data.jld", "data", monkeys)

### 讀入已儲存的訓練資料

In [4]:
train_x = load("monkey_data.jld")["data"]
train_x = reshape(train_x, 72, 128, 3, :)
train_x = Float32.(train_x)

72×128×3×1097 Array{Float32,4}:
[:, :, 1, 1] =
 0.415686  0.215686    0.545098  …  0.345098    0.329412  0.580392
 0.156863  1.0         0.254902     0.156863    0.784314  0.0509804
 0.223529  0.239216    0.537255     0.184314    0.772549  0.270588
 1.0       0.466667    0.4          0.647059    0.262745  0.54902
 0.219608  0.25098     0.223529     0.976471    0.184314  0.815686
 0.427451  1.0         0.156863  …  0.388235    0.513726  0.54902
 0.501961  0.368627    0.101961     0.137255    0.207843  0.239216
 0.243137  0.235294    0.254902     0.32549     0.219608  0.815686
 0.576471  0.690196    0.419608     0.490196    0.168627  0.666667
 0.960784  0.745098    0.380392     0.509804    0.615686  0.576471
 0.466667  0.72549     0.337255  …  0.00392157  0.141176  0.447059
 0.407843  0.376471    0.466667     0.560784    0.356863  0.223529
 0.796078  0.415686    0.431373     0.94902     0.878431  0.0823529
 ⋮                               ⋱  ⋮                     
 0.654902  0.682353    

### 載入驗證資料

In [7]:
val_monkeys = Array{Float64}(undef, 272, 72, 128, 3)
val_key = 1
pbase = "validation/validation/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = imresize(img, (288, 512))
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(72, 128, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:72)
                for k in (1:128)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        val_monkeys[val_key, :, :, :] = c2
        val_key += 1
    end
end

### 儲存驗證資料

In [9]:
save("monkey_validation.jld", "data", val_monkeys)

### 讀入已儲存的驗證資料

In [5]:
test_x = load("monkey_validation.jld")["data"]
test_x = reshape(test_x, 72, 128, 3, :)
test_x = Float32.(test_x)

72×128×3×272 Array{Float32,4}:
[:, :, 1, 1] =
 0.423529   0.552941   0.0862745  …  0.290196   0.0588235  0.0705882
 0.521569   0.368627   0.494118      0.482353   0.145098   0.454902
 0.352941   0.0745098  0.47451       0.768627   0.0431373  0.333333
 0.113725   0.0901961  0.227451      0.164706   0.0470588  0.415686
 0.25098    0.0235294  0.231373      0.196078   0.858824   0.682353
 0.0862745  0.333333   0.172549   …  0.309804   0.435294   0.188235
 0.827451   0.603922   0.478431      0.0862745  0.0470588  0.737255
 0.709804   0.0156863  0.364706      0.564706   0.572549   0.439216
 0.921569   0.309804   0.686275      0.290196   0.435294   0.176471
 0.568627   0.239216   0.223529      0.337255   0.2        0.207843
 0.823529   0.670588   0.505882   …  0.227451   0.0901961  0.454902
 0.6        0.898039   0.223529      0.258824   0.396078   0.196078
 0.984314   0.65098    0.521569      0.129412   0.85098    0.423529
 ⋮                                ⋱  ⋮                     
 0.607843

### 訓練標籤

In [6]:
train_labels = Array{Int32}(undef, 1097)
key = 1
amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
for i in (0:9)
    for j in (1:amount[i + 1])
        train_labels[key] = i
        key += 1
    end
end
train_y = onehotbatch(train_labels, 0:9)

10×1097 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

### 驗證標籤

In [7]:
val_labels = Array{Int32}(undef, 272)
key = 1
amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
for i in (0:9)
    for j in (1:amount[i + 1])
        val_labels[key] = i
        key += 1
    end
end
test_y = onehotbatch(val_labels, 0:9)

10×272 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [8]:
batchsize = 256
train = DataLoader(train_x, train_y, batchsize = batchsize, shuffle = true)
test = DataLoader(test_x, test_y, batchsize = batchsize)

DataLoader((Float32[0.42352942 0.5529412 … 0.05882353 0.07058824; 0.52156866 0.36862746 … 0.14509805 0.45490196; … ; 0.12941177 0.16078432 … 0.1254902 0.25490198; 0.22352941 0.90588236 … 0.18431373 0.1882353]

Float32[0.34901962 0.36862746 … 0.32156864 0.25490198; 0.34117648 0.4 … 0.011764706 0.5921569; … ; 0.08627451 0.06666667 … 0.69411767 0.56078434; 0.21960784 0.34509805 … 0.36078432 0.1254902]

Float32[0.52156866 0.48235294 … 0.25882354 0.58431375; 0.5058824 0.6862745 … 0.29411766 0.24313726; … ; 0.9098039 0.6313726 … 0.05882353 0.6313726; 0.4862745 0.22745098 … 0.4392157 0.4]

Float32[0.16470589 0.85490197 … 0.75686276 0.4509804; 0.2 0.29411766 … 0.42745098 0.37254903; … ; 0.12156863 0.25882354 … 0.4745098 0.41960785; 0.6117647 0.4627451 … 0.050980393 0.627451]

Float32[0.08627451 0.05882353 … 0.3019608 0.3647059; 0.79607844 0.2 … 0.5019608 0.2901961; … ; 0.45490196 1.0 … 0.02745098 0.007843138; 0.6666667 0.16470589 … 0.38431373 0.41960785]

Float32[0.29803923 0.32156864 … 0.2784

## CNN 模型

In [9]:
model = Chain(
    # Block 1
    Conv((3, 3), 3 => 32, pad = (1, 1), relu),
    Conv((3, 3), 32 => 32, pad = (1, 1), relu),
    Dropout(0.3),
    Conv((3, 3), 32 => 64, pad = (1, 1), relu),
    MaxPool((2, 2)),
    
    # Block 2
    Conv((3, 3), 64 => 128, pad = (1, 1), relu),
    Dropout(0.3),
    Conv((3, 3), 128 => 128, pad = (1, 1), relu),
    MaxPool((2, 2)),
    
    # Block 3
    Conv((3, 3), 128 => 256, pad = (1, 1), relu),
    Dropout(0.4),
    Conv((3, 3), 256 => 256, pad = (1, 1), relu),
    Dropout(0.4),
    Conv((3, 3), 256 => 256, pad = (1, 1), relu),
    MaxPool((2, 2)),
    
    # Flatten & Dense
    flatten,
    Dense(36864, 256),
    Dense(256, 128),
    Dense(128, 10),
    softmax)

Chain(Conv((3, 3), 3=>32, relu), Conv((3, 3), 32=>32, relu), Dropout(0.3), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 64=>128, relu), Dropout(0.3), Conv((3, 3), 128=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 128=>256, relu), Dropout(0.4), Conv((3, 3), 256=>256, relu), Dropout(0.4), Conv((3, 3), 256=>256, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(36864, 256), Dense(256, 128), Dense(128, 10), softmax)

## 損失函數

In [10]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

## Callback 函式

In [11]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l / length(test)
end

test_loss (generic function with 1 method)

In [12]:
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

## 模型訓練

In [13]:
epochs = 30
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb = throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0


┌ Info: Epoch 2
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0


┌ Info: Epoch 3
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0


┌ Info: Epoch 29
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0


┌ Info: Epoch 30
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0
test_loss() = 2.4084158f0


## 模型評估

In [14]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [16]:
accuracy(test_x, test_y)

0.09926470588235294